In [1]:
!pip install sentencepiece kaggle datasets
# !mkdir ~/.kaggle
# !mv kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets download balabaskar/harry-potter-books-corpora-part-1-7
# !unzip -u harry-potter-books-corpora-part-1-7.zip
# !rm /content/characters_list.csv
# !rm /content/harry-potter-books-corpora-part-1-7.zip
!mkdir -p /content/data/
# !mv -f Book* /content/data/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from datasets import load_dataset
import numpy as np
import torch
from torch import nn
import pandas as pd
import sentencepiece as spm
from typing import *
from collections import defaultdict

In [3]:
dataset = load_dataset("wikitext", 'wikitext-103-raw-v1')

  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
LIMIT = 50_000

with open('/content/data/train.txt', 'w') as f:
  for line in dataset['train']['text'][:LIMIT]:
    f.write(line + '\n')

# with open('/content/data/test.txt', 'w') as f:
#   for line in dataset['test']['text'][:LIMIT]:
#     f.write(line + '\n')

# with open('/content/data/validation.txt', 'w') as f:
#   for line in dataset['validation']['text'][:LIMIT]:
#     f.write(line + '\n')

In [5]:
spm.SentencePieceTrainer.train('--input=/content/data/train.txt --model_prefix=m --vocab_size=5000')

# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [6]:
def encode(sent: str) -> List[int]:
  return sp.encode_as_ids(sent)

def decode(tok_idxs: List[int]) -> str:
  return sp.decode(tok_idxs)

In [7]:
train_data = ""
with open('/content/data/train.txt', 'r') as f:
  for line in f:
    train_data += line

val_data = ""
with open('/content/data/validation.txt', 'r') as f:
  for line in f:
    val_data += line


train_data = (encode(train_data))
val_data = (encode(val_data))

In [8]:
N_GRAM = 10
VOCAB_SIZE = sp.get_piece_size()
print(f'Vocab size: {VOCAB_SIZE}')

def get_batch(batch_size: int, n_gram: int, data: str = 'train') -> List[List[int]]:
  data = train_data if data =="train" else val_data
  start_idx = torch.randint(len(data) - (n_gram -1) , (batch_size, ))
  pred_index = start_idx + n_gram
  xs = torch.stack([data[i:(i+n_gram)] for i in start_idx])
  ys = data[pred_index].view(-1,1)
  return(xs, ys, start_idx)

class NGramModel(object):
  def __init__(self, n_gram: int) -> None:
    self.n = n_gram
    self.grams = defaultdict(lambda: 0)

  def train(self, train_data: List[int]) -> None:
    # get count of all n-grams
    for i in range(len(train_data[:-self.n])):
      gram = train_data[i:i+self.n]
      self.grams[tuple(gram)] += 1

    # get count of all n-1 grams that are in the n-grams
    n_minus_1_grams = defaultdict(lambda: 0)
    for i in range(len(train_data[:-(self.n-1)])):
      gram = train_data[i:i+(self.n-1)]
      n_minus_1_grams[tuple(gram)] += 1

    for gram in self.grams.keys():
      self.grams[gram] /= n_minus_1_grams[gram[:-1]]

  def generate(self, context: List[int], length: int) -> None:
    assert len(context) == self.n - 1, 'Context length is not the same as the n-gram size.'
    res = []
    res.extend(context)

    for _ in range(length):


      probs = [self.grams[gram] for gram in self.grams.keys() if \
               tuple(context) == gram[:-1]]
      vals = [gram[-1] for gram in self.grams.keys() if \
               tuple(context) == gram[:-1]]
      next_token = np.random.choice(vals, 1, p=probs)

      res.append(int(next_token))

      context.append(int(next_token))
      context.pop(0)

    print(decode(res))

mod = NGramModel(N_GRAM)
mod.train(train_data)

Vocab size: 5000


In [10]:
start = np.random.choice(np.arange(len(list(mod.grams.keys()))))
context = list(list(mod.grams.keys())[start][:-1])
mod.generate(context, 100)

AMISOM allies . With a change in administration in late 2010 , government troops and their military partners had succeeded in forcing out Al @-@ Shabaab by August 2011 . Mogadishu has subsequently experienced a period of intense reconstruction spearheaded by the Somali diaspora , the municipal authorities , and Turkey , a historic ally of Somalia . = = = Reconstruction = = = In August 2011 , militant group al
